In [18]:
import requests
from PIL import Image

import torch
from transformers import LlavaForConditionalGeneration, AutoProcessor

In [20]:
model_id = '/Users/jykim/Desktop/new/mlc-imp/dist/models/llava-1.5-7b-hf'

image_file = "http://images.cocodataset.org/val2017/000000039769.jpg"
prompt = "USER: <image>\nWhat are these?\nASSISTANT:"
processor = AutoProcessor.from_pretrained(model_id)

raw_image = Image.open(requests.get(image_file, stream=True).raw)
inputs = processor(prompt, raw_image, return_tensors='pt')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
model = LlavaForConditionalGeneration.from_pretrained(
    model_id, 
    torch_dtype=torch.float16, 
    low_cpu_mem_usage=True, 
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [27]:
model.vision_tower(inputs['pixel_values'])

RuntimeError: "LayerNormKernelImpl" not implemented for 'Half'

In [13]:
clip_encoder = model.vision_tower.vision_model.encoder

In [14]:
traced_model = torch.jit.trace(clip_encoder, )

In [16]:
exported_mod = export(clip_encoder, torch.randn(1, 577, 1024))

ModuleNotFoundError: No module named 'torch._export'